In [45]:
# Import the dependencies.
import pandas as pd
import requests
# Import the API key.
from config import g_key
  # Make request and get the JSON data from the search.
import json
import gmaps

In [29]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs),Country,Date
0,0,Kodiak,57.79,-152.41,19.40,48,20,38.21,few clouds,0.00,0.0,US,2020-02-22 06:23:01
1,1,Vaini,-21.20,-175.20,80.60,78,75,13.87,smoke,0.00,0.0,TO,2020-02-22 06:26:40
2,2,Port Alfred,-33.59,26.89,69.01,90,96,5.01,light rain,0.19,0.0,ZA,2020-02-22 06:26:42
3,3,Busselton,-33.65,115.33,81.00,40,100,9.24,overcast clouds,0.00,0.0,AU,2020-02-22 06:26:47
4,4,Punta Arenas,-53.15,-70.92,53.60,81,79,14.99,broken clouds,0.00,0.0,CL,2020-02-22 06:25:06


In [30]:
#Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_amount = input("Do you want it to be raining? (yes/no) ")
snow_amount = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 70
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [31]:
# Filter the dataset to find the cities that fit the criteria.
if rain_amount == "no" and snow_amount == "no":
        prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                        (city_data_df["Max Temp"] >= min_temp) &
                                        (city_data_df["Rain inches (last 3 hrs)"] == 0) &
                                        (city_data_df["Snow inches (last 3 hrs)"] == 0)]

elif rain_amount == "no" and snow_amount == "yes":
        prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                        (city_data_df["Max Temp"] >= min_temp) &
                                        (city_data_df["Rain inches (last 3 hrs)"] == 0) &
                                        (city_data_df["Snow inches (last 3 hrs)"] > 0.0)]

preferred_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs),Country,Date
6,6,Lebu,-37.62,-73.65,49.96,98,4,1.86,clear sky,0.0,0.0,CL,2020-02-22 06:22:57
15,15,Glendive,47.11,-104.71,33.80,74,1,12.75,clear sky,0.0,0.0,US,2020-02-22 06:27:38
26,26,Nikolskoye,59.70,30.79,37.99,74,75,17.90,broken clouds,0.0,0.0,RU,2020-02-22 06:27:49
31,31,Fortuna,40.60,-124.16,48.99,93,90,1.23,overcast clouds,0.0,0.0,US,2020-02-22 06:23:16
37,37,Dmitriyevskaya,45.66,40.77,38.98,80,100,14.67,overcast clouds,0.0,0.0,RU,2020-02-22 06:27:58


In [32]:
preferred_cities_df.count()

City_ID                     114
City                        114
Lat                         114
Lng                         114
Max Temp                    114
Humidity                    114
Cloudiness                  114
Wind Speed                  114
Description                 114
Rain inches (last 3 hrs)    114
Snow inches (last 3 hrs)    114
Country                     114
Date                        114
dtype: int64

In [39]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Lebu,CL,49.96,clear sky,-37.62,-73.65,
15,Glendive,US,33.80,clear sky,47.11,-104.71,
26,Nikolskoye,RU,37.99,broken clouds,59.70,30.79,
31,Fortuna,US,48.99,overcast clouds,40.60,-124.16,
37,Dmitriyevskaya,RU,38.98,overcast clouds,45.66,40.77,
40,Krasnogorsk,RU,34.00,broken clouds,55.82,37.32,
41,Obodivka,UA,30.00,few clouds,48.41,29.25,
48,Sabha,LY,49.30,clear sky,27.04,14.43,
55,Kargopol,RU,32.67,light snow,61.51,38.95,
64,Stromness,GB,37.40,light snow,58.97,-3.30,


In [40]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [42]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
        
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("success")
    except IndexError as e:
        print(e)

success
success
success
success
list index out of range
success
list index out of range
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
list index out of range
success
list index out of range
success
success
success
list index out of range
success
success
success
success
success
success
success
list index out of range
success
success
list index out of range
success
list index out of range
list index out of range
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
list index out of range
success
success
success
success
success
success
success
success
list index out of range
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
list ind

In [43]:
hotel_df

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
6,Lebu,CL,49.96,clear sky,-37.62,-73.65,Hostal Las Lilas
15,Glendive,US,33.80,clear sky,47.11,-104.71,Super 8 by Wyndham Glendive
26,Nikolskoye,RU,37.99,broken clouds,59.70,30.79,Tourist House - Sablino
31,Fortuna,US,48.99,overcast clouds,40.60,-124.16,Super 8 By Wyndham Fortuna
37,Dmitriyevskaya,RU,38.98,overcast clouds,45.66,40.77,
...,...,...,...,...,...,...,...
534,Bereda,ES,46.99,fog,43.27,-7.54,Pazo de Verdes
535,Barbastro,ES,43.00,clear sky,42.04,0.13,Hotel San Ramón del Somontano
536,Shepetivka,UA,32.79,scattered clouds,50.19,27.06,Turhenyev
544,Rincon,US,37.00,clear sky,32.30,-81.24,Bent tree subdivision


In [58]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [59]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description</dt><dd>{Description}</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [53]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")